In [ ]:
# handle missing values
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")
housing_num = housing.select_dtypes(include=[np.number])
imputer.fit(housing_num)
# transform the trainig set
X = imputer.transform(housing_num) 

In [ ]:
# drop outliers
from sklearn.ensemble import IsolationForest

isolation_forest = IsolationForest(random_state=42)
outlier_pred = isolation_forest.fit_predict(X)
housing = housing.iloc[outlier_pred == 1]
housing_labels = housing_labels.iloc[outlier_pred == 1]

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

#ordinal encoding is used for categorical features with ordered categories
ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)
#check encodied categories
ordinal_encoder.categories_

from sklearn.preprocessing import OneHotEncoder
#one hot encoding is used for categorical features with no ordered categories
cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
cat_encoder.categories_

## Feature Scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler
# use a range
min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
housing_num_min_max_scaled = min_max_scaler.fit_transform(housing_num)

from sklearn.preprocessing import StandardScaler
#use mean and std
std_scaler = StandardScaler()
housing_num_std_scaled = std_scaler.fit_transform(housing_num)

# consider using log for power law distributions with long tails or buketize 

In [ ]:
# multinomial features (have multiple peaks), generate new features using rbf kernel
from sklearn.metrics.pairwise import rbf_kernel

age_simil_35 = rbf_kernel(housing[["housing_median_age"]], [[35]], gamma=0.1)

## Feature Engineering:
Interaction and poloynomial features 

In [1]:
# Import Packages

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import silhouette_score, silhouette_samples,f1_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler,OneHotEncoder,LabelEncoder
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer

import optuna
import scipy.cluster
from sklearn.pipeline import Pipeline

import datetime


from sklearn.model_selection import cross_val_score, train_test_split
from catboost import CatBoostClassifier

from sklearn.feature_selection import SelectKBest, f_classif


In [2]:
df= pd.read_csv("credit_default.csv")
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   UserID             6000 non-null   object
 1   Sex                6000 non-null   object
 2   PreviousDefault    6000 non-null   int64 
 3   FirstName          6000 non-null   object
 4   LastName           6000 non-null   object
 5   NumberPets         6000 non-null   int64 
 6   PreviousAccounts   6000 non-null   int64 
 7   ResidenceDuration  6000 non-null   int64 
 8   Street             6000 non-null   object
 9   LicensePlate       6000 non-null   object
 10  BadCredit          6000 non-null   int64 
 11  Amount             6000 non-null   int64 
 12  Married            6000 non-null   int64 
 13  Duration           6000 non-null   int64 
 14  City               6000 non-null   object
 15  Purpose            6000 non-null   object
 16  DateOfBirth        6000 non-null   object


,UserID,Sex,PreviousDefault,FirstName,LastName,NumberPets,PreviousAccounts,ResidenceDuration,Street,LicensePlate,BadCredit,Amount,Married,Duration,City,Purpose,DateOfBirth
0,218-84-8180,F,0,Debra,Schaefer,2,3,1,503 Linda Locks,395C,0,3907,0,24,Port Keith,Vacation,1964-04-07
1,395-49-9764,M,0,Derek,Wright,0,1,1,969 Cox Dam Suite 101,UFZ 691,0,3235,0,12,Lake Debra,NewCar,1978-06-02
2,892-81-4890,F,0,Shannon,Smith,0,0,2,845 Kelly Estate,48A•281,0,3108,1,30,North Judithbury,NewCar,1972-03-18
3,081-11-7963,F,0,Christina,Brooks,2,1,3,809 Burns Creek,30Z J39,1,4014,1,36,Lake Chad,Other,1985-02-26
4,347-03-9639,M,0,Ralph,Anderson,1,5,1,248 Brandt Plains Apt. 465,71-Q331,1,3823,0,18,North Judithbury,Vacation,1983-08-08


In [3]:
X = df.drop('BadCredit', axis=1) #.select_dtypes(['number'])
y = df['BadCredit']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Baseline Model

In [4]:

# Columns to drop and to apply one-hot encoding
columns_to_drop = ['UserID','FirstName','LastName','Street','LicensePlate','DateOfBirth']
columns_to_encode = ['Sex','City','Purpose']

# Column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('drop', 'drop', columns_to_drop),
        ('onehot', OneHotEncoder(), columns_to_encode)
    ],
    remainder='passthrough'  # keeps the columns not specified in transformers
)
# Define the CatBoost model
catboost_model = CatBoostClassifier(verbose=False)
# Create pipeline
pipeline_1 = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', catboost_model)])


# Perform cross-validation and then calculate scores
scores = cross_val_score(pipeline_1, X_train, y_train, cv=5,scoring="f1")

# Output the mean accuracy
print(f"Mean CV F1: {scores.mean():.2f}")


Mean CV F1: 0.36


### Adding feature engineering
1. Age baesd on DOB
2. Interaction varaiable based on duration and amount
3. standard scaler on all numerical columns